In [ ]:
! pip install -U transformers datasets evaluate seqeval

In [ ]:
!pip install accelerate -U

In [ ]:
! pip install transformers[torch]

In [ ]:
import datasets

In [ ]:
from datasets import Dataset

In [ ]:
ds = Dataset.from_json('data.json')

In [ ]:
nerdataset = datasets.DatasetDict({'train': ds})

In [ ]:
label_listner = ['0','DOMAIN_NAME','GEO_LOCATION','URL','MALWARE_NAME','HASH_VAL','ACTOR','FILE_PATH','IP','REG_KEY','EMAIL_ADDRESSES']

In [ ]:
label_listnerI = ['0','DOMAIN_NAME','GEO_LOCATION','URL','MALWARE_NAME','HASH_VAL','ACTOR','FILE_PATH','IP','REG_KEY','EMAIL_ADDRESSES']

In [ ]:
procesData = []
for i in range(len(nerdataset["train"])):
  ctext = nerdataset["train"][i]['text']
  texttokens = ctext.split()
  dic = {"text": texttokens,"annotations":[]}

  count = 0
  for f in range(len(texttokens)):
    if f == 0:#在句首用B
      if count == len(nerdataset["train"][i]['annotations']):
        dic['annotations'].append(0)
      else:
        if texttokens[f] == nerdataset["train"][i]['annotations'][count]['entity']:
          index = label_listner.index(nerdataset["train"][i]['annotations'][count]['label'])
          dic['annotations'].append(index)
          count = count + 1
        else:
          dic['annotations'].append(0)
    else:
      if dic['annotations'][-1] != 0:#上一个tag不等于0用I
        if count == len(nerdataset["train"][i]['annotations']):
          dic['annotations'].append(0)
        else:
          if texttokens[f] == nerdataset["train"][i]['annotations'][count]['entity']:
            index = label_listnerI.index(nerdataset["train"][i]['annotations'][count]['label'])
            index = index + len(label_listnerI)
            dic['annotations'].append(index)
            count = count + 1
          else:
            dic['annotations'].append(0)
      else:#上一个tag等于0用B
        if count == len(nerdataset["train"][i]['annotations']):
          dic['annotations'].append(0)
        else:
          if texttokens[f] == nerdataset["train"][i]['annotations'][count]['entity']:
            index = label_listner.index(nerdataset["train"][i]['annotations'][count]['label'])
            dic['annotations'].append(index)
            count = count + 1
          else:
            dic['annotations'].append(0)

  procesData.append(dic)

In [ ]:
import json

In [ ]:
json.dumps(procesData)

In [ ]:
pds = Dataset.from_json('nerdata.json')

In [ ]:
pnerdataset = datasets.DatasetDict({'train': pds})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"annotations"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_pnerdataset = pnerdataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
label_listnerBI = ["O","B-DOMAIN_NAME","B-GEO_LOCATION","B-URL","B-MALWARE_NAME","B-HASH_VAL","B-ACTOR","B-FILE_PATH","B-IP","B-REG_KEY","B-EMAIL_ADDRESSES","I-DOMAIN_NAME","I-GEO_LOCATION","I-URL","I-MALWARE_NAME","I-HASH_VAL","I-ACTOR","I-FILE_PATH","I-IP","I-REG_KEY","I-EMAIL_ADDRESSES"]

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_listnerBI[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_listnerBI[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2labelner = {
    0: "O",
    1: "B-DOMAIN_NAME",
    2: "B-GEO_LOCATION",
    3: "B-URL",
    4: "B-MALWARE_NAME",
    5: "B-HASH_VAL",
    6: "B-ACTOR",
    7: "B-FILE_PATH",
    8: "B-IP",
    9: "B-REG_KEY",
    10: "B-EMAIL_ADDRESSES",
    11: "I-DOMAIN_NAME",
    12: "I-GEO_LOCATION",
    13: "I-URL",
    14: "I-MALWARE_NAME",
    15: "I-HASH_VAL",
    16: "I-ACTOR",
    17: "I-FILE_PATH",
    18: "I-IP",
    19: "I-REG_KEY",
    20: "I-EMAIL_ADDRESSES",
}
label2idner = {
    "O": 0,
    "B-DOMAIN_NAME": 1,
    "B-GEO_LOCATION": 2,
    "B-URL": 3,
    "B-MALWARE_NAME": 4,
    "B-HASH_VAL": 5,
    "B-ACTOR": 6,
    "B-FILE_PATH": 7,
    "B-IP": 8,
    "B-REG_KEY": 9,
    "B-EMAIL_ADDRESSES": 10,
    "I-DOMAIN_NAME": 11,
    "I-GEO_LOCATION": 12,
    "I-URL": 13,
    "I-MALWARE_NAME": 14,
    "I-HASH_VAL": 15,
    "I-ACTOR": 16,
    "I-FILE_PATH": 17,
    "I-IP": 18,
    "I-REG_KEY": 19,
    "I-EMAIL_ADDRESSES": 20,

}

In [ ]:
small_train_datasetT = tokenized_pnerdataset["train"].shuffle(seed=42).select(range(64))
small_eval_datasetT = tokenized_pnerdataset["train"].shuffle(seed=42).select(range(16))

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=21, id2label=id2labelner, label2id=label2idner
)

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/trainModel",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_datasetT,
    eval_dataset=small_eval_datasetT,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
metric = load_metric("seqeval")

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_pnerdataset["train"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [id2labelner[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [id2labelner[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results